In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../top_rated_wines.csv')
df.head()

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."


In [9]:
df.describe()
print(df.columns)

print(df.name[0])

Index(['name', 'region', 'variety', 'rating', 'notes'], dtype='object')
3 Rings Reserve Shiraz 2004


In [13]:
# Now let's create easy formatting for creating the embeddings
# Use records keyword to make it look like a json file (key, value pairs based on the column names)
# reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html
df[0:1].to_dict('records')

[{'name': '3 Rings Reserve Shiraz 2004',
  'region': 'Barossa Valley, Barossa, South Australia, Australia',
  'variety': 'Red Wine',
  'rating': 96.0,
  'notes': 'Vintage Comments : Classic Barossa vintage conditions. An average wet Spring followed by extreme heat in early February. Occasional rainfall events kept the vines in good balance up to harvest in late March 2004. Very good quality coupled with good average yields. More than 30 months in wood followed by six months tank maturation of the blend prior to bottling, July 2007. '}]

In [29]:
# Find rows that are empty
print(len(df))
df.isnull()
null_data = df[df.isnull().any(axis=1)]
print(len(null_data))

notnull_data = df[df.notnull().all(axis=1)]
print(len(notnull_data))

1365
18
1347


In [46]:
# Sample 100 rows in this dataframe
# Changes each time unless you specify a random state
#df.sample(100).to_dict('records')
df = pd.read_csv('../../top_rated_wines.csv')
df = df[df['variety'].notna()]

data = df.to_dict('records')
print(len(data))

data = df.sample(100).to_dict('records')
print(len(data))
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

1347
100


In [34]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')
# Models downloaded locally

In [36]:
# Create vector database client
# instance of the database
qdrant = QdrantClient(":memory:")

In [37]:
# create a collection to store the wine data
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # size of vector defined by model
        distance=models.Distance.COSINE  # we will eval embeddings with cosine sim
    )
)


True

In [48]:
# Then, vectorize the wine data! Upload the data points to the collection we just made

qdrant.upload_records(
    collection_name="top_wines",
    records=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc                     
        ) for idx, doc in enumerate(data)
    ]
)

In [59]:
# Now we can search the database

hits = qdrant.search(
    collection_name="top_wines",
    #query_vector=encoder.encode("99 points Cabernet Sauvignon from Napa Valley").tolist(),
    query_vector=encoder.encode("Spicy Red Wine").tolist(),
    limit=3
)

for hit in hits:
    print(f"\nScore:{hit.score:.2f} Matched Payload {hit.payload}")


Score:0.67 Matched Payload {'name': 'Greenock Creek Apricot Block Shiraz 2002', 'region': 'Barossa Valley, Barossa, South Australia, Australia', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'This rich, dark red wine has complex aromas of spice and fruit, plums, berries and a splash of dark chocolate with a lingering velvety finish. '}

Score:0.67 Matched Payload {'name': 'Greenock Creek Apricot Block Shiraz 2000', 'region': 'Barossa Valley, Barossa, South Australia, Australia', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'This rich, dark red wine has complex aromas of spice and fruit, plums, berries and a splash of dark chocolate with a lingering velvety finish.'}

Score:0.60 Matched Payload {'name': 'Bruna Grimaldi Barolo Badarina 2013', 'region': 'Barolo, Piedmont, Italy', 'variety': 'Red Wine', 'rating': 97.0, 'notes': "This wine's color is an intense and brilliant garnet with orange highlights. The aroma is ample and intense, with sensations of violets, pepper, raspberries,

In [ ]:
user_prompt = "Suggest me a spicy red wine."


hits = qdrant.search(
    collection_name="top_wines",
    #query_vector=encoder.encode("99 points Cabernet Sauvignon from Napa Valley").tolist(),
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)


In [63]:
# Store search results in variable
search_results=[hit.payload for hit in hits]

In [62]:
#for hit in hits:
#    print(f"\nScore:{hit.score:.2f} Matched Payload {hit.payload}")

from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8000/v1",
    api_key="sk-no-key-required"
    
)


In [65]:
# Use local large langugage model
completion=client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role":"system","content":"You are a chatbot. You are a wine specialist. Your top priority is to help the user choose a wine."},
        {"role":"user","content": user_prompt},
        {"role":"assistant", "content": str(search_results)}
    
    ]
)

nt(completion.choices[0].message)

APIConnectionError: Connection error.